In [1]:
import shadow_reconstruction
import qutip as qt
import numpy
from matplotlib import pyplot
import os

In [2]:
def make_perms(n):
    return itertools.permutations(numpy.arange(n))

def gen_cliffords():
    yield from qt.qubit_clifford_group()


def get_num_cons_cliffords():
    def f(x):
        y = numpy.eye(4, dtype=numpy.complex128)
        y[1:3, 1:3] = x
        return y
    return {
        k: f(v.__array__()) for k, v in enumerate(gen_cliffords())
    }

In [3]:
cliffords = numpy.array(list(get_num_cons_cliffords().values()))

/tmp/ipykernel_46992/2038288818.py:5: DeprecationWarning: Importing functions/classes of the qip submodule directly from the namespace qutip is deprecated. Please import them from the submodule instead, e.g.
from qutip.qip.operations import cnot
from qutip.qip.circuit import QubitCircuit

  yield from qt.qubit_clifford_group()


# Simple to start

$$ \vert \psi \rangle = \frac{|01\rangle + |10\rangle}{\sqrt{2}}$$

In [4]:
zz = numpy.array([[0.,1.,1.,0.]])
zz /= numpy.linalg.norm(zz)

rho = zz.T @ zz
# pyplot.imshow(rho)
# pyplot.show()

print(numpy.trace(rho))
rho = shadow_reconstruction.DensityMatrix.new_mixed_sparse(rho.astype(numpy.complex128))

0.9999999999999998


# Bigger

$$|\psi\rangle = \frac{|1000> + |0001>}{\sqrt{2}}$$

In [5]:
# (01 + 10)00
zz = numpy.zeros((1, 1<<4), dtype=numpy.complex128)
zz[0, 0b1000] = 1.0
zz[0, 0b0100] = 1.0
zz /= numpy.linalg.norm(zz)
rho = zz.T @ zz
# print(numpy.trace(rho))
rho /= numpy.trace(rho)
rho = shadow_reconstruction.DensityMatrix.new_mixed_sparse(rho.astype(numpy.complex128))
rho.expectation_string('ZIII'), rho.expectation_string('ZZII')

(0j, (-1+0j))

In [6]:
rho = shadow_reconstruction.DensityMatrix.new_pure_dense(zz[0,:].astype(numpy.complex128))
rho.expectation_string('ZIII'), rho.expectation_string('ZZII')

(0j, (-0.9999999999999998+0j))

In [7]:
exp = shadow_reconstruction.Experiment(4, ops=cliffords)

In [8]:
# if os.path.exists('larger_samples_pure.dat'):
#     print("Loading...")
#     samples = shadow_reconstruction.Samples.load_from('larger_samples_pure.dat')
# else:
print("Sampling...")
samples = exp.sample(rho, 1_000_000)
print("Saving...")

Sampling...
Saving...


In [9]:
recon = shadow_reconstruction.Reconstruction(False)

In [10]:
recon.use_smart_estimator()
recon.estimate_string("ZIII", samples), rho.expectation_string('ZIII')

((-3.640000000000367e-05+0j), 0j)

In [11]:
recon.use_smart_estimator()
recon.estimate_string("ZZII", samples), rho.expectation_string('ZZII')

((-1.0009116000000033+0j), (-0.9999999999999998+0j))

In [12]:
recon.use_smart_estimator()
recon.estimate_string("ZIZI", samples), rho.expectation_string('ZIZI')

((0.0004481999999999992+0j), 0j)

In [13]:
recon.use_smart_estimator()
recon.estimate_string("+-II", samples), rho.expectation_string('+-II')

((0.5004404999999998+0.003154499999999999j), (0.4999999999999999+0j))

In [14]:
recon.use_smart_estimator()
recon.estimate_string("-++-", samples), rho.expectation_string('-++-')

(0j, 0j)

# Check filtering

In [23]:
recon = shadow_reconstruction.Reconstruction(False)
meas = numpy.round(recon.estimate_string_for_each_sample("+-II", samples).real, decimals=5)
vals, counts = numpy.unique(meas, return_counts=True)
vals, counts, sum(counts), numpy.var(meas)

(array([0. , 4.5]), array([888791, 111209]), 1000000, 2.0015415559597494)

In [24]:
recon.estimate_string("+-II", samples)

(0.5004404999999998+0.003154499999999999j)

In [25]:
recon = shadow_reconstruction.Reconstruction(True)
meas = numpy.round(recon.estimate_string_for_each_sample("+-II", samples).real, decimals=5)
vals, counts = numpy.unique(meas, return_counts=True)
vals, counts, sum(counts), numpy.var(meas)

(array([0. , 1.5]), array([222697, 111209]), 333906, 0.4997909343329443)

In [26]:
recon.estimate_string("+-II", samples)

(0.49958221774990547+0.0031490898636143093j)